# Mirror 1

In [1]:
import numpy as np
import pandas as pd

# === USER INPUTS ===
wavelength_nm = 675               # Incident wavelength in nm
aoi_deg = 45                   # Angle of incidence in degrees
n_complex = 1.7328 + 1j * 7.9374      # Aluminum index of refraction at 675 nm

# Convert angle to radians
aoi_rad = np.radians(aoi_deg)
n1 = 1.0  # Ambient medium (air)

# === Fresnel coefficients for metallic mirror ===
theta_i = aoi_rad
theta_t = np.arcsin(n1 * np.sin(theta_i) / n_complex)  # Snell's Law (complex)

cos_theta_i = np.cos(theta_i)
cos_theta_t = np.sqrt(1 - (n1 * np.sin(theta_i) / n_complex)**2)

# Fresnel reflection coefficients (complex)
rs = ((n1 * cos_theta_i - n_complex * cos_theta_t) /
      (n1 * cos_theta_i + n_complex * cos_theta_t))
rp = ((n_complex * cos_theta_i - n1 * cos_theta_t) /
      (n_complex * cos_theta_i + n1 * cos_theta_t))

# Reflectance (real)
R_s = np.abs(rs)**2
R_p = np.abs(rp)**2

# Phase shift between r_p and r_s
delta = np.angle(rp) - np.angle(rs)

# === Optionally allow user to override R_s, R_p, delta ===
# Uncomment and set these if you want to use measured values instead of calculated ones
# R_s = ...
# R_p = ...
# delta = ...

# Fresnel reflection coefficients (complex, example values)
# r_s = 0.992 * np.exp(1j * np.deg2rad(12.918))
# r_p = 0.975 * np.exp(-1j * np.deg2rad(0.751))

# Define the 3x3 extended Jones matrix
J_ext = np.array([
    [rs, 0,   0],
    [0,   0,   1],
    [0,  -rp, 0]
])

# Extract the intrinsic 2x2 Jones matrix from the (s, p) -> (s', p') mapping
J = np.array([
    [J_ext[0, 0], J_ext[0, 1]],
    [J_ext[2, 0], J_ext[2, 1]]
])

# Define Pauli matrices
sigma = [
    np.array([[1, 0], [0, 1]]),             # sigma_0
    np.array([[1, 0], [0, -1]]),            # sigma_1
    np.array([[0, 1], [1, 0]]),             # sigma_2
    np.array([[0, -1j], [1j, 0]])           # sigma_3
]

# Compute Mueller matrix using Pauli-trace formula
M = np.zeros((4, 4), dtype=float)
for i in range(4):
    for j in range(4):
        M[i, j] = 0.5 * np.real(np.trace(sigma[i] @ J @ sigma[j] @ J.conj().T))

# Display Mueller matrix as a copy-pasteable numpy array
print("Mueller Matrix from Extended Jones Matrix (copy-paste as np.array):")
with np.printoptions(precision=5, suppress=True):
    print("np.array([")
    for row in M:
        print("    [" + ", ".join(f"{v:.5f}" for v in row) + "],")
    print("])")

# Compute and print retardance, diattenuation, and bulk rotation

def compute_diattenuation(M):
    return np.abs(M[0, 1]) / M[0, 0]

def compute_retardance(M):
    m22 = M[2:4, 2:4]
    trace = np.trace(m22)
    return np.real(np.arccos(trace / 2))

def compute_rotation_angle(M):
    return 0.5 * np.degrees(np.arctan2(M[1, 0], M[0, 0]))

ret = compute_retardance(M)
dia = compute_diattenuation(M)
rot = compute_rotation_angle(M)

M1 = M.copy()

print(f"Retardance:     {ret / (2 * np.pi):.4f} waves")
print(f"Diattenuation:  {dia:.6f}")
print(f"Rotation Angle: {rot:.2f} degrees")

Mueller Matrix from Extended Jones Matrix (copy-paste as np.array):
np.array([
    [0.89694, 0.03270, 0.00000, 0.00000],
    [0.03270, 0.89694, 0.00000, 0.00000],
    [0.00000, 0.00000, 0.88355, -0.15093],
    [0.00000, 0.00000, 0.15093, 0.88355],
])
Retardance:     0.0776 waves
Diattenuation:  0.036459
Rotation Angle: 1.04 degrees


# Mirror 2

In [2]:
import numpy as np
import pandas as pd

# === USER INPUTS ===
wavelength_nm = 675               # Incident wavelength in nm
aoi_deg = 10                      # Angle of incidence in degrees
n_complex = 1.7328 + 1j * 7.9374      # Aluminum index of refraction at 675 nm

# Convert angle to radians
aoi_rad = np.radians(aoi_deg)
n1 = 1.0  # Ambient medium (air)

# === Fresnel coefficients for metallic mirror ===
theta_i = aoi_rad
theta_t = np.arcsin(n1 * np.sin(theta_i) / n_complex)  # Snell's Law (complex)

cos_theta_i = np.cos(theta_i)
cos_theta_t = np.sqrt(1 - (n1 * np.sin(theta_i) / n_complex)**2)

# Fresnel reflection coefficients (complex)
rs = ((n1 * cos_theta_i - n_complex * cos_theta_t) /
      (n1 * cos_theta_i + n_complex * cos_theta_t))
rp = ((n_complex * cos_theta_i - n1 * cos_theta_t) /
      (n_complex * cos_theta_i + n1 * cos_theta_t))

# Reflectance (real)
R_s = np.abs(rs)**2
R_p = np.abs(rp)**2

# Phase shift between r_p and r_s
delta = np.angle(rp) - np.angle(rs)

# === Optionally allow user to override R_s, R_p, delta ===
# Uncomment and set these if you want to use measured values instead of calculated ones
# R_s = ...
# R_p = ...
# delta = ...

# Fresnel reflection coefficients (complex, example values)
# r_s = 0.992 * np.exp(1j * np.deg2rad(12.918))
# r_p = 0.975 * np.exp(-1j * np.deg2rad(0.751))

# Define the 3x3 extended Jones matrix
J_ext = np.array([
    [0, 1,   0],
    [-rp,   0,   1],
    [0,  0, rs]
])

# Extract the intrinsic 2x2 Jones matrix from the (s, p) -> (s', p') mapping
J = np.array([
    [J_ext[0, 0], J_ext[0, 1]],
    [J_ext[2, 0], J_ext[2, 1]]
])

# Define Pauli matrices
sigma = [
    np.array([[1, 0], [0, 1]]),             # sigma_0
    np.array([[1, 0], [0, -1]]),            # sigma_1
    np.array([[0, 1], [1, 0]]),             # sigma_2
    np.array([[0, -1j], [1j, 0]])           # sigma_3
]

# Compute Mueller matrix using Pauli-trace formula
M = np.zeros((4, 4), dtype=float)
for i in range(4):
    for j in range(4):
        M[i, j] = 0.5 * np.real(np.trace(sigma[i] @ J @ sigma[j] @ J.conj().T))

# Display Mueller matrix as a copy-pasteable numpy array
print("Mueller Matrix from Extended Jones Matrix (copy-paste as np.array):")
with np.printoptions(precision=5, suppress=True):
    print("np.array([")
    for row in M:
        print("    [" + ", ".join(f"{v:.5f}" for v in row) + "],")
    print("])")

# Compute and print retardance, diattenuation, and bulk rotation

def compute_diattenuation(M):
    return np.abs(M[0, 1]) / M[0, 0]

def compute_retardance(M):
    m22 = M[2:4, 2:4]
    trace = np.trace(m22)
    return np.real(np.arccos(trace / 2))

def compute_rotation_angle(M):
    return 0.5 * np.degrees(np.arctan2(M[1, 0], M[0, 0]))

ret = compute_retardance(M)
dia = compute_diattenuation(M)
rot = compute_rotation_angle(M)

M2 = M.copy()

print(f"Retardance:     {ret / (2 * np.pi):.4f} waves")
print(f"Diattenuation:  {dia:.6f}")
print(f"Rotation Angle: {rot:.2f} degrees")

Mueller Matrix from Extended Jones Matrix (copy-paste as np.array):
np.array([
    [0.50000, -0.50000, 0.00000, 0.00000],
    [0.50000, -0.50000, 0.00000, 0.00000],
    [0.00000, 0.00000, 0.00000, 0.00000],
    [0.00000, 0.00000, 0.00000, 0.00000],
])
Retardance:     0.2500 waves
Diattenuation:  1.000000
Rotation Angle: 22.50 degrees


# Combined Mirrors

In [3]:
M3 = M1 @ M2

# Display Mueller matrix as a copy-pasteable numpy array
print("Mueller Matrix from Extended Jones Matrix (copy-paste as np.array):")
with np.printoptions(precision=5, suppress=True):
    print("np.array([")
    for row in M3:
        print("    [" + ", ".join(f"{v:.5f}" for v in row) + "],")
    print("])")

ret = compute_retardance(M3)
dia = compute_diattenuation(M3)
rot = compute_rotation_angle(M3)

print(f"Retardance:     {ret / (2 * np.pi):.4f} waves")
print(f"Diattenuation:  {dia:.6f}")
print(f"Rotation Angle: {rot:.2f} degrees")

Mueller Matrix from Extended Jones Matrix (copy-paste as np.array):
np.array([
    [0.46482, -0.46482, 0.00000, 0.00000],
    [0.46482, -0.46482, 0.00000, 0.00000],
    [0.00000, 0.00000, 0.00000, 0.00000],
    [0.00000, 0.00000, 0.00000, 0.00000],
])
Retardance:     0.2500 waves
Diattenuation:  1.000000
Rotation Angle: 22.50 degrees
